### Расчётно-графическая работа №4
*Выполнил студент группы R3341 Овчинников Павел (вариант №2.2)*

---


In [1]:
# Импортируем необходимые модули
import pandas as pd
import numpy as np

from scipy import stats

## Задание №1
Требуется построить линейную модель (предполагая нормальность распределения ошибок, некоррелированность компонент, гомоскедастичность), вычислить оценки коэффициентов модели и остаточной дисперсии, построить для них доверительные интервалы, вычислить коэффициент детерминации, проверить указанные в условии гипотезы с помощью построенной линейной модели.

В датасете представлены данные о мобильных телефонах.
1. Построить линейную модель, где в качестве независимых переменных выступают высота и ширина экрана (атрибуты `sc_h` и `sc_w`) и ёмкость аккумулятора (атрибут `battery_power`), зависимой — масса телефона (`mobile_wt`).
2. Проверить следующие подозрения:
    - Чем больше высота экрана, тем больше масса
    - Чем больше ширина экрана, тем больше масса
    - Проверьте гипотезу $H_0$ об одновременном равенстве нулю коэффициентов при ширине экрана и ёмкости аккумулятора против альтернативы $H_1 = \overline{H_0}$.

Для начала считаем датасет и извлечём необходимые нам столбцы данных, с которыми мы будем работать.

In [2]:
data = pd.read_csv('mobile_phones.csv')
data.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1


In [3]:
h, w, p, m = data['sc_h'], data['sc_w'], data['battery_power'], data['mobile_wt']

Итак, как в нашем случае будет выглядеть линейная модель:
$$m_i = b_0 + h_ib_1 + w_ib_2 + p_ib_3 + \xi_i$$
Здесь $m_i$ - масса телефона, $h_i$ - высота экрана, $w_i$ - ширина экрана, $p_i$ - ёмкость аккумулятора, $\xi_i$ - случайная ошибка, $b_0, b_1, b_2, b_3$ - коэффициенты модели, а $i$ - номер наблюдения $0,\ldots n$.
При этом ошибка $\xi_i$ удовлетворяет следующим условиям:
1. $E(\xi_i) = 0$
2. $D(\xi_i) = \sigma^2$
3. $cov(\xi_i, \xi_j) = 0, i \neq j$

Но мы воспользуемся представлением модели в матричной форме:
$$Y = Xb + E$$
Здесь $Y$ - вектор значений зависимой переменной, $X$ - матрица значений независимых переменных или т.н. регрессоров, $b$ - вектор коэффициентов модели, $E$ - вектор случайных ошибок.  
При этом матрица $X$ имеет вид:
$$X = \begin{bmatrix}
\vdots & \vdots & \vdots & \vdots \\
1 & h_i & w_i & p_i \\
\vdots & \vdots & \vdots & \vdots
\end{bmatrix}$$
При этом, пользуясь методом наименьших квадратов, оценим вектор коэффициентов $b$ следующим образом:
$$\hat{b} = (X^TX)^{-1}X^TY$$

In [4]:
X = np.array([np.ones(len(data)), h, w, p]).T
Y = m
b = np.linalg.inv(X.T @ X) @ X.T @ Y

np.set_printoptions(precision=5)
np.set_printoptions(suppress=True)
print('b:', b)

b: [143.64057  -0.26355  -0.03955   0.00006]


Получаем оценку уравнения регрессии:
$$\hat{m} = 143.64057 - 0.26355 h - 0.03955 w + 0.00006 p$$

Теперь рассчитаем остаточную дисперсию:
$$\hat{\sigma}^2 = \frac{1}{n - k} \sum_{i=1}^{n} \hat{\xi}_i^2 = \frac{\varepsilon^T\varepsilon}{n - k}$$
где $n$ - количество наблюдений, $k$ - количество коэффициентов модели (в нашем случае $k = 4$), $\varepsilon = Y - \hat{Y} = Y - X\hat{b}$ - вектор остатков.

In [5]:
k, n = len(b), len(data)
e = Y - X @ b
s2 = (e.T @ e) / (n - k)  # type: ignore
print(f's2: {s2:.5f}')

s2: 1253.55763


Теперь посчитаем ковариационную матрицу оценок коэффициентов модели:
$$\hat{\Sigma}_{\hat{b}} = \hat{\sigma}^2(X^TX)^{-1}$$

In [6]:
np.set_printoptions(precision=6)

covb = np.linalg.inv(X.T @ X) * s2  # type: ignore
covb

array([[11.293254, -0.461523,  0.026398, -0.004149],
       [-0.461523,  0.047517, -0.023241,  0.000009],
       [ 0.026398, -0.023241,  0.044426,  0.000003],
       [-0.004149,  0.000009,  0.000003,  0.000003]])

Теперь, используя диагональные элементы ковариационной матрицы, вычислим доверительные интервалы и стандартные ошибки для коэффициентов модели:
$$\left(\hat{b}_i - SE(\hat{b}_i)t_{1-\frac{\alpha}2}(n-k);\hat{b}_i + SE(\hat{b}_i)t_{1-\frac{\alpha}2}(n-k)\right)$$
Здесь $t_{1-\frac{\alpha}2}(n-k)$ - квантиль распределения Стьюдента с $n - k$ степенями свободы уровня значимости $\frac{\alpha}2$, а $SE(\hat{b}_i)$ - стандартная ошибка оценки коэффициента модели, вычисляющаяся следующим образом:
$$SE(\hat{b}_i) = \sqrt{\hat{\Sigma}_{ii}}$$

In [7]:
alpha = 0.05
t = stats.t.ppf(1 - alpha / 2, n - k)
print('№\tЗначение\tОшибка\t\tДоверительный интервал')
for i in range(k):
    se = np.sqrt(covb[i, i])
    left, right = b[i] - t * se, b[i] + t * se
    print(f'{i}\t{b[i]:.6f}\t{se:.6f}\t({left:.6f}; {right:.6f})')

№	Значение	Ошибка		Доверительный интервал
0	143.640572	3.360544	(137.050031; 150.231113)
1	-0.263548	0.217983	(-0.691046; 0.163950)
2	-0.039549	0.210774	(-0.452908; 0.373811)
3	0.000064	0.001803	(-0.003471; 0.003600)


Теперь рассчитаем доверительный интервал для дисперсии остатков:
$$\left(\frac{(n - k)\hat{\sigma}^2}{\chi^2_{1-\frac{\alpha}2}(n-k)};\frac{(n - k)\hat{\sigma}^2}{\chi^2_{\frac{\alpha}2}(n-k)}\right)$$
Здесь $\chi^2_{1-\frac{\alpha}2}(n-k)$ и $\chi^2_{\frac{\alpha}2}(n-k)$ - квантили распределения хи-квадрат с $n - k$ степенями свободы уровней значимости $1 - \frac{\alpha}2$ и $\frac{\alpha}2$ соответственно.

In [8]:
base = (n - k) * s2
left = base / stats.chi2.ppf(1 - alpha / 2, n - k)
right =  base / stats.chi2.ppf(alpha / 2, n - k)
print('s2\t\tДоверительный интервал')
print(f'{s2:.5f}\t({left:.5f}; {right:.5f})')

s2		Доверительный интервал
1253.55763	(1179.28052; 1335.11605)


Наконец, вычислим коэффициент детерминации:
$$R^2 = 1 - \frac{\sum\hat{\xi}_i^2}{\sum(y_i - \overline{y})^2} = 1 - \frac{\varepsilon^T\varepsilon}{(Y - \overline{Y})^T(Y - \overline{Y})}

In [9]:
R2 = 1 - (e.T @ e) / ((Y - Y.mean()) @ (Y - Y.mean()))  # type: ignore
print(f'{R2:.5f}')

0.00116



---

Проверим подозрения. Для этого зададим гипотезы к каждому подозрению и проверим их с помощью p-value.  
Для этого воспользуемся статистикой:
$$t_{stat} = \frac{\hat{b}_i}{SE(\hat{b}_i)}$$

### Гипотеза №1
- $H_0$: $b_1 = 0$, т.е. высота экрана не влияет на массу телефона
- $H_1$: $b_1 > 0$, т.е. высота экрана прямо пропорционально влияет на массу телефона

In [10]:
t_stat = b[1] / np.sqrt(covb[1, 1])
p_value = 1 - stats.t.cdf(t_stat, n - k)
print('Гипотеза H0 отвергается' if p_value < alpha else 'Гипотеза H0 принимается')

Гипотеза H0 принимается


### Гипотеза №2
- $H_0$: $b_2 = 0$, т.е. ширина экрана не влияет на массу телефона
- $H_1$: $b_2 > 0$, т.е. ширина экрана прямо пропорционально влияет на массу телефона

In [11]:
t_stat = b[2] / np.sqrt(covb[2, 2])
p_value = 1 - stats.t.cdf(t_stat, n - k)
print('Гипотеза H0 отвергается' if p_value < alpha else 'Гипотеза H0 принимается')

Гипотеза H0 принимается


### Гипотеза №3
- $H_0$: $b_2 = 0 \cap b_3 = 0$, т.е. ширина экрана и ёмкость аккумулятора не влияют на массу телефона
- $H_1$: $b_2 \neq 0 \cup b_3 \neq 0$, т.е. хотя бы одна из переменных влияет на массу телефона

Проверим с помощью F-теста. Статистика имеет вид:
$$F = \frac{\frac{\text{RSS}_S - \text{RSS}}{q}}{\frac{\text{RSS}}{n - k}} = \frac{(\text{RSS}_S - \text{RSS})(n - k)}{\text{RSS}q} = \frac{\text{RSS}_S - \text{RSS}}{\text{RSS}}\frac{n - k}{q} \sim F(q, n-k)$$
Здесь $\text{RSS}_S$ - сумма квадратов остатков сокращённой модели, $q$ - количество ограничений в модели (фактически, $k - k_S$), $\text{RSS}$ - сумма квадратов остатков полной модели, $n$ - количество наблюдений, $k$ - количество коэффициентов модели. $RSS = \varepsilon^T\varepsilon$ и вычислялось ранее, при оценке дисперсии остатков.

Суть метода состоит в том, что мы получаем сокращённую модель, зависимую только от высоты экрана, зануляя коэффициенты при ширине экрана и ёмкости аккумулятора, согласно гипотезе $H_0$.  
После этого выполним проверку F-тестом. Если $F > F_{1-\alpha}(q, n-k)$, то отвергнем нулевую гипотезу.

In [12]:
X_S = np.array([np.ones(len(data)), h]).T
b_S = np.linalg.inv(X_S.T @ X_S) @ X_S.T @ Y
e_S = Y - X_S @ b_S
RSS_S = e_S.T @ e_S
RSS = e.T @ e
q = k - len(b_S)

f_score = ((RSS_S - RSS) / RSS) * ((n - k) / q)  # type: ignore
print(f'{f_score:.5f}')

0.01829


Значение статистики получено. Теперь найдём критическое значение для уровня значимости $\alpha = 0.05$, числа ограничений  $q = 2$ и числа степеней свободы $n - k$.

In [13]:
f = stats.f.ppf(1 - alpha, q, n - k)
print('Гипотеза H0 отвергается' if f_score > f else 'Гипотеза H0 принимается')

Гипотеза H0 принимается


### Вывод
- Коэффициент детерминации мал и близок к нулю, т.е. модель статистически значимо не объясняет зависимость зависимой переменной от независимых, между переменными слабая связь,
- Первое подозрение не подтвердилось, т.е. высота экрана не влияет на массу телефона,
- Второе подозрение не подтвердилось, т.е. ширина экрана не влияет на массу телефона,
- Третья гипотеза подтвердилась, т.е. ни одна из двух переменных ширины и ёмкости аккумулятора не влияет на массу телефона.

## Задание №2
Проверить гипотезу о равенстве средних на каждом уровне фактора с помощью модели однофакторного дисперсионного анализа.

В датасете представлены данные о сдаче экзаменов. Фактор — этническая/национальная группа. Выходная переменная — суммарный балл за все 3 экзамена.

Но для начала, по традиции, считаем датасет и извлечём необходимые нам столбцы данных, с которыми мы будем работать.

In [14]:
data = pd.read_csv('exams_dataset.csv')
data.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,high school,standard,none,67,91,84
1,female,group D,some college,standard,none,63,63,67
2,female,group A,associate's degree,standard,completed,73,83,85
3,female,group C,associate's degree,standard,none,85,98,95
4,male,group B,bachelor's degree,standard,none,75,57,63


Посмотрим, какие в датасете представлены группы и сколько их, а также сформируем столбец с выходной переменной:

In [15]:
print(data['race/ethnicity'].unique())
data['total score'] = data['math score'] + data['reading score'] + data['writing score']
data.head()

['group B' 'group D' 'group A' 'group C' 'group E']


,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score,total score
0,female,group B,high school,standard,none,67,91,84,242
1,female,group D,some college,standard,none,63,63,67,193
2,female,group A,associate's degree,standard,completed,73,83,85,241
3,female,group C,associate's degree,standard,none,85,98,95,278
4,male,group B,bachelor's degree,standard,none,75,57,63,195


Теперь сгруппируем данные по группам и посчитаем необходимые нам статистики:

In [16]:
grouped_data = data.groupby('race/ethnicity')['total score'].agg(['mean', 'count'])
grouped_data

,mean,count
race/ethnicity,,
group A,191.662338,77
group B,195.063725,204
group C,194.981481,324
group D,209.536398,261
group E,223.805970,134


### Гипотеза
- $H_0$: $\mu_1 = \mu_2 = \ldots = \mu_k$, т.е. средние значения на всех уровнях фактора равны
- $H_1$: $\exists\  i,j: \mu_i \neq \mu_j$, т.е. хотя бы на одном уровне фактора среднее значение отличается от остальных

Воспользуемся F-статистикой:
$$F = \frac{\frac{\text{SSB}}{k - 1}}{\frac{\text{SSW}}{n - k}} = \frac{\text{SSB}(n - k)}{\text{SSW}(k - 1)} \sim F(k - 1, n - k)$$
Здесь $\text{SSB}$ - сумма квадратов для межгрупповой дисперсии, $\text{SSW}$ - сумма квадратов для внутригрупповой дисперсии, $k$ - количество групп, $n$ - общее количество наблюдений.
$$\text{SSB} = \sum_{j = 1}^{k} n_j(\overline{y}_j - \overline{y})^2\qquad \text{SSW} = \sum_{j=1}^{k}\sum_{i=1}^{n_j}(y_{ij} - \overline{y}_j)^2$$

In [17]:
n, k = len(data), len(grouped_data)
data_mean = data['total score'].mean()
SSB = (grouped_data['count'] * (grouped_data['mean'] - data_mean) ** 2).sum()
SSW = data.groupby('race/ethnicity')['total score'].apply(lambda group: sum((group - group.mean()) ** 2)).sum()
F = SSB * (n - k) / (SSW * (k - 1))

p_value = 1 - stats.f.cdf(F, k - 1, n - k)

print(f'F:\t\t{F:.5f}')
print(f'p-value:\t{p_value:.5f}')
print('Гипотеза H0 отвергается' if p_value < alpha else 'Гипотеза H0 принимается')

F:		14.31081
p-value:	0.00000
Гипотеза H0 отвергается


### Вывод
Гипотеза о равенстве средних на всех уровнях фактора отвергается, т.е. хотя бы на одном уровне фактора среднее значение отличается от остальных.